# 0) Install & Login & Drive Mount

In [1]:
!pip install -q wandb polars xgboost imblearn

In [2]:
import wandb
wandb.login()
# 계정 만들었으면, 2 누르고 본인 api 키 입력

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice:

 2


wandb: You chose 'Use an existing W&B account'
wandb: Logging into https://api.wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: Create a new API key at: https://wandb.ai/authorize?ref=models
wandb: Store your API key securely and do not share it.
wandb: Paste your API key and hit enter:

 ··········


wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: 0326byeol (0326byeol-korea-ac-kr) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 1) Imports

In [4]:
import polars as pl
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import RobustScaler, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

from sklearn.metrics import (
    roc_auc_score, roc_curve,
    average_precision_score, precision_recall_curve,  # AUPRC/PR curve 핵심
    confusion_matrix, classification_report,
    accuracy_score, precision_score, recall_score, f1_score
)

from xgboost import XGBClassifier
from imblearn.over_sampling import SMOTE
from collections import Counter
import random

import joblib
import pickle
import os

# 2) 테이블 병합(Polars + Parquet)



In [5]:
# 파일 경로 설정 (본인의 경로에 맞게 수정하세요)
base_path = "/content/drive/MyDrive/data"
parquet_dir = os.path.join(base_path, "parquet")
os.makedirs(parquet_dir, exist_ok=True)

trans_csv = os.path.join(base_path, "HI-Medium_Trans.csv")
acc_csv   = os.path.join(base_path, "HI-Medium_accounts.csv")

In [6]:
# Parquet 저장 폴더
trans_parquet = os.path.join(parquet_dir, "HI-Medium_Trans.parquet")
acc_parquet   = os.path.join(parquet_dir, "HI-Medium_accounts.parquet")
master_parquet = os.path.join(parquet_dir, "HI-Medium_Master.parquet")

In [7]:
# (처음 1회만) CSV -> Parquet 변환(스트리밍으로 처리되어 메모리 안전)
#    - 이미 parquet가 있으면 스킵
if not os.path.exists(trans_parquet):
    pl.scan_csv(
        trans_csv,
        infer_schema_length=10000,
        try_parse_dates=False
    ).sink_parquet(trans_parquet)

if not os.path.exists(acc_parquet):
    pl.scan_csv(
        acc_csv,
        infer_schema_length=10000,
    ).sink_parquet(acc_parquet)

In [8]:
# Lazy 모드로 CSV 연결
q_trans = pl.scan_parquet(trans_parquet)
q_acc   = pl.scan_parquet(acc_parquet)

In [9]:
#   Rates LazyFrame
#   - Currency_Rate.csv를 따로 scan_csv해도 되는데,
#     지금은 고정 테이블로 만든다고 했으니 lazy DF로 구성
# -----------------------------
rates_df = pl.DataFrame({
    "currency": [
        "US Dollar","Euro","UK Pound","Swiss Franc","Canadian Dollar",
        "Australian Dollar","Bitcoin","Saudi Riyal","Shekel","Brazil Real",
        "Yuan","Mexican Peso","Ruble","Rupee","Yen"
    ],
    "rate": [
        1.0,0.99,1.15,1.03,0.76,
        0.67,20000.0,0.266,0.29,0.19,
        0.14,0.05,0.017,0.0125,0.007
    ]
}).lazy()


In [10]:
# -----------------------------
#  컬럼명 normalize (데이터마다 Account/Account.1 케이스가 있어서 방어)
# -----------------------------
# trans의 실제 컬럼을 먼저 확인하고 싶으면:
# print(q_trans.columns)  # Lazy에서도 가능

q_trans = (
    q_trans
    # 만약 컬럼명이 'Account', 'Account_duplicated_0'이라고 가정하면:
    .rename({
        "Account": "From Account",      # 첫 번째 Account는 보내는 사람
        "Account_duplicated_0": "To Account"       # 두 번째(중복) Account는 받는 사람
        # ※ 주의: 위에서 print(q_trans.columns)로 확인한 실제 이름과 다르면
        #         여기를 실제 이름에 맞춰 수정해야 합니다!
    })
)

# 이제 trans는 최소한 From Account / To Account 컬럼이 있다고 가정하고 진행

In [11]:
# -----------------------------
#  Master build
#   1) Receiving Currency로 Amount Received USD
#   2) Payment Currency로 Amount Paid USD
#   3) accounts를 sender/receiver로 2번 left join (suffix 처리)
# -----------------------------
q_master = (
    q_trans

    # --- (1) Amount Received USD (Receiving Currency 기준)
    .join(
        rates_df.rename({"currency": "Receiving Currency", "rate": "recv_rate"}),
        on="Receiving Currency",
        how="left"
    )
    .with_columns([
        (pl.col("recv_rate").fill_null(1.0)).alias("recv_rate"),
        (pl.col("Amount Received") * pl.col("recv_rate")).alias("Amount_Received_USD")
    ])

    # --- (2) Amount Paid USD (Payment Currency 기준)
    .join(
        rates_df.rename({"currency": "Payment Currency", "rate": "pay_rate"}),
        on="Payment Currency",
        how="left"
    )
    .with_columns([
        (pl.col("pay_rate").fill_null(1.0)).alias("pay_rate"),
        (pl.col("Amount Paid") * pl.col("pay_rate")).alias("Amount_Paid_USD")
    ])

    # --- (3) Sender account join
    .join(
        q_acc.select(["Account Number", "Entity Name", "Bank Name"]),
        left_on="From Account",
        right_on="Account Number",
        how="left",
        suffix="_sender"
    )
    .rename({
        "Entity Name": "Sender_Entity",
        "Bank Name": "Sender_Bank_Name"
    })

    # --- (4) Receiver account join (컬럼 충돌 방지 위해 suffix 사용)
    .join(
        q_acc.select(["Account Number", "Entity Name", "Bank Name"]),
        left_on="To Account",
        right_on="Account Number",
        how="left",
        suffix="_receiver"
    )
    .rename({
        "Entity Name": "Receiver_Entity",
        "Bank Name": "Receiver_Bank_Name"
    })

    # (선택) rate 컬럼 정리하고 싶으면:
   .drop(["recv_rate","pay_rate"])
)

In [12]:
# -----------------------------
#  sink to Parquet (Streaming write)
# -----------------------------
print("Writing master table to Parquet (streaming)...")
q_master.sink_parquet(master_parquet)
print(f"Done. Saved: {master_parquet}")

# 이후에는 이렇게 빠르게 로드
# q_master = pl.scan_parquet(master_parquet)

Writing master table to Parquet (streaming)...
Done. Saved: /content/drive/MyDrive/data/parquet/HI-Medium_Master.parquet


In [13]:
master_parquet = "/content/drive/MyDrive/data/parquet/HI-Medium_Master.parquet"

q_master = pl.scan_parquet(master_parquet)

# 4) Timestamp

In [14]:
# =========================================================
# Timestamp 파싱 + 정렬 준비
#   - 원본 Timestamp(string) 유지
#   - ts(Datetime) 새로 생성
#   - ts 파싱 실패(null) 개수 출력
# =========================================================
total_rows = q_master.select(pl.len()).collect().item()

q_master = q_master.with_columns(
    pl.col("Timestamp")
      .str.strptime(pl.Datetime, format="%Y/%m/%d %H:%M", strict=False)
      .alias("ts")
)

null_ts_rows = (
    q_master.filter(pl.col("ts").is_null())
     .select(pl.len())
     .collect()
     .item()
)

valid_rows = total_rows - null_ts_rows

print(f"Total rows                    : {total_rows:,}")
print(f"Rows with NULL ts (parse fail): {null_ts_rows:,}")
print(f"Valid ts rows                 : {valid_rows:,}")

# split은 ts가 유효한 행만 대상으로 진행 (추천)
q_valid = q_master.filter(pl.col("ts").is_not_null())

Total rows                    : 31,898,669
Rows with NULL ts (parse fail): 0
Valid ts rows                 : 31,898,669


In [15]:
# =========================================================
# time-based split 기준(ts cut) 계산 (60:20:20)
#   - collect 최소화: 경계 ts 2개만 뽑음
# =========================================================
n = q_valid.select(pl.len()).collect().item()
train_end_idx = int(n * 0.6)
val_end_idx   = int(n * 0.8)

q_sorted = q_valid.sort("ts")

train_cut_ts = (
    q_sorted.select("ts")
            .slice(train_end_idx, 1)
            .collect()
            .item()
)

val_cut_ts = (
    q_sorted.select("ts")
            .slice(val_end_idx, 1)
            .collect()
            .item()
)

print("Train cutoff ts:", train_cut_ts)
print("Val cutoff ts  :", val_cut_ts)

Train cutoff ts: 2022-09-09 20:43:00
Val cutoff ts  : 2022-09-14 05:46:00


In [16]:
# =========================================================
# split 컬럼 부여 (train/val/test)
# =========================================================
q_split = q_sorted.with_columns(
    pl.when(pl.col("ts") <= train_cut_ts)
      .then(pl.lit("train"))
      .when(pl.col("ts") <= val_cut_ts)
      .then(pl.lit("val"))
      .otherwise(pl.lit("test"))
      .alias("split")
)

# sanity check: split 분포
split_counts = (
    q_split.group_by("split")
           .agg(pl.len().alias("rows"))
           .collect()
)
print(split_counts)

# 라벨 비율 sanity check (optional)
label_counts = (
    q_split.group_by(["split", "Is Laundering"])
           .agg(pl.len().alias("rows"))
           .collect()
)
print(label_counts)

shape: (3, 2)
┌───────┬──────────┐
│ split ┆ rows     │
│ ---   ┆ ---      │
│ str   ┆ u32      │
╞═══════╪══════════╡
│ test  ┆ 6378958  │
│ train ┆ 19139731 │
│ val   ┆ 6379980  │
└───────┴──────────┘
shape: (6, 3)
┌───────┬───────────────┬──────────┐
│ split ┆ Is Laundering ┆ rows     │
│ ---   ┆ ---           ┆ ---      │
│ str   ┆ i64           ┆ u32      │
╞═══════╪═══════════════╪══════════╡
│ train ┆ 0             ┆ 19124195 │
│ val   ┆ 1             ┆ 9054     │
│ test  ┆ 0             ┆ 6368318  │
│ val   ┆ 0             ┆ 6370926  │
│ train ┆ 1             ┆ 15536    │
│ test  ┆ 1             ┆ 10640    │
└───────┴───────────────┴──────────┘


# 6) bucket_ts 생성 (1h)

In [17]:
# =========================================================
# bucket_ts 생성 (ts → 1h truncate)
#   - baseline v1: 거래(row) 단위 모델링이지만,
#     이후 확장을 위해 bucket_ts는 만들어 둠
# =========================================================
q_feat = q_split.with_columns(
    pl.col("ts").dt.truncate("1h").alias("bucket_ts")
).with_columns(
    pl.col("bucket_ts").dt.strftime("%Y-%m-%d %H:%M:%S").alias("bucket_ts_str")
)

# bucket 분포 sanity check (optional)
bucket_check = (
    q_feat.filter(pl.col("split") == "train")
          .select([
              pl.col("bucket_ts").min().alias("train_bucket_min"),
              pl.col("bucket_ts").max().alias("train_bucket_max"),
              pl.len().alias("train_rows"),
          ])
          .collect()
)
print(bucket_check)

shape: (1, 3)
┌─────────────────────┬─────────────────────┬────────────┐
│ train_bucket_min    ┆ train_bucket_max    ┆ train_rows │
│ ---                 ┆ ---                 ┆ ---        │
│ datetime[μs]        ┆ datetime[μs]        ┆ u32        │
╞═════════════════════╪═════════════════════╪════════════╡
│ 2022-09-01 00:00:00 ┆ 2022-09-09 20:00:00 ┆ 19139731   │
└─────────────────────┴─────────────────────┴────────────┘


In [18]:
#“노드 키” 만들기 (송신/수신 각각)
q_feat = q_feat.with_columns([
    (pl.col("From Account").cast(pl.Utf8) + "_" + pl.col("bucket_ts_str").cast(pl.Utf8)).alias("sender_node"),
    (pl.col("To Account").cast(pl.Utf8)   + "_" + pl.col("bucket_ts_str").cast(pl.Utf8)).alias("receiver_node"),
])

# 8) 피처 엔지니어링

In [19]:
# -------------------------
# (A) 상수/룩업 리스트
# -------------------------
PAYMENT_FORMATS = ["ACH", "Cheque", "Bitcoin", "Cash", "Credit Card", "Wire", "Reinvestment"]

HIGH_RISK_SENDER_BANKS = [
    "National Bank of Dallas", "Savings Bank of Augusta", "China Bank #27",
    "India Bank #96", "Brazil Bank #128", "National Bank of Milford",
    "Savings Bank of Sacramento", "Saudi Arabia Bank #14", "Israel Bank #16",
    "Golden Credit Union"
]

HIGH_RISK_RECEIVER_BANKS = [
    "China Bank #292", "China Bank #27", "China Bank #22", "Japan Bank #143",
    "Brazil Bank #50", "Bank of Denver", "Saudi Arabia Bank #56",
    "Israel Bank #48", "The Pine Bank", "National Bank of Milford"
]

# 엔티티 타입 가중치 (v1: 단순 룰)
ENTITY_TYPE_SCORE_MAP = {
    "Corporation": 2.0,
    "Sole Proprietorship": 1.5
}

# Payment method risk (v1: 단순 룰)
# (정확한 가중치는 EDA에서 조정 가능. 지금은 “비중 높았던 수단에 가중”만 구현)
PAYMENT_RISK_MAP = {
    "ACH": 3.0,
    "Bitcoin": 2.5,
    "Cheque": 2.5,
    "Cash": 1.5,
    "Credit Card": 1.3,
    "Wire": 1.0,
    "Reinvestment": 1.0
}

In [20]:
# -------------------------
# (B) Timestamp 파생
# -------------------------
q_feat = q_feat.with_columns([
    pl.col("ts").dt.hour().cast(pl.Int16).alias("hour"),
    pl.col("ts").dt.weekday().cast(pl.Int8).alias("day_of_week"),  # 월=1 ~ 일=7 (Polars 기본)
    pl.col("ts").dt.weekday().is_in([6, 7]).alias("is_weekend"),
    pl.col("ts").dt.date().alias("ts_day"),

    # 새벽 여부 (예: 0~5시)
    pl.col("ts").dt.hour().is_between(0, 5).alias("is_dawn"),

    # TimeofDay bucket (예시: 새벽/오전/오후/밤)
    pl.when(pl.col("ts").dt.hour().is_between(0, 5)).then(pl.lit("dawn"))
      .when(pl.col("ts").dt.hour().is_between(6, 11)).then(pl.lit("morning"))
      .when(pl.col("ts").dt.hour().is_between(12, 17)).then(pl.lit("afternoon"))
      .otherwise(pl.lit("evening"))
      .alias("timeofday_bucket"),
])

In [21]:
# -------------------------
# (C) Amount 파생 (float32 + log)
# -------------------------
q_feat = q_feat.with_columns([
    # float32 캐스팅
    pl.col("Amount_Paid_USD").cast(pl.Float32),
    pl.col("Amount_Received_USD").cast(pl.Float32),

    # log1p
    pl.col("Amount_Paid_USD").log1p().alias("log_amount_paid_usd"),
    pl.col("Amount_Received_USD").log1p().alias("log_amount_received_usd"),

    # Round number (1000 단위)
    (pl.col("Amount_Paid_USD") % 1000 == 0).alias("is_round_1000_paid"),
    (pl.col("Amount_Received_USD") % 1000 == 0).alias("is_round_1000_received"),

    # 더 강한 라운드 (10000 단위)
    (pl.col("Amount_Paid_USD") % 10000 == 0).alias("is_round_10000_paid"),
])

In [22]:
# -------------------------
# (D) Payment Format 원핫 + 파생
# -------------------------
# 원핫(각 format별 bool)
q_feat = q_feat.with_columns([
    (pl.col("Payment Format") == "ACH").alias("is_ach"),
    (pl.col("Payment Format") == "Cheque").alias("is_cheque"),
    (pl.col("Payment Format") == "Bitcoin").alias("is_bitcoin_fmt"),
    (pl.col("Payment Format") == "Cash").alias("is_cash"),
    (pl.col("Payment Format") == "Credit Card").alias("is_credit_card"),
    (pl.col("Payment Format") == "Wire").alias("is_wire"),
    (pl.col("Payment Format") == "Reinvestment").alias("is_reinvestment"),
])

# Crypto 여부 (format이 Bitcoin이거나 currency가 Bitcoin이면 True)
q_feat = q_feat.with_columns([
    ((pl.col("Payment Format") == "Bitcoin") | (pl.col("Payment Currency") == "Bitcoin"))
      .alias("is_crypto_transfer"),

    # High value ACH (>= 1,000,000 USD & ACH)
    ((pl.col("Payment Format") == "ACH") & (pl.col("Amount_Paid_USD") >= 1_000_000))
      .alias("is_high_value_ach"),
])

# Payment_Method_Risk (format 기반 가중치)
q_feat = q_feat.with_columns([
    pl.col("Payment Format")
      .replace(PAYMENT_RISK_MAP, default=1.0)
      .cast(pl.Float32)
      .alias("payment_method_risk"),
])

# format × amount interaction (v1용)
q_feat = q_feat.with_columns([
    (pl.col("payment_method_risk") * pl.col("log_amount_paid_usd")).alias("risk_x_log_paid"),
    (pl.col("is_ach").cast(pl.Int8) * pl.col("log_amount_paid_usd")).alias("ach_x_log_paid"),
    (pl.col("is_bitcoin_fmt").cast(pl.Int8) * pl.col("log_amount_paid_usd")).alias("btc_x_log_paid"),
])

In [23]:
# -------------------------
# (E) Entity type 파싱 + 점수
# -------------------------
# "Corporation #26522" → "Corporation"
q_feat = q_feat.with_columns([
    pl.col("Sender_Entity")
      .cast(pl.Utf8)
      .str.replace(r"\s*#\d+.*$", "")   # "#숫자" 이후 제거
      .alias("sender_entity_type"),

    pl.col("Receiver_Entity")
      .cast(pl.Utf8)
      .str.replace(r"\s*#\d+.*$", "")
      .alias("receiver_entity_type"),
])

# 타입 점수
q_feat = q_feat.with_columns([
    pl.col("sender_entity_type")
      .replace(ENTITY_TYPE_SCORE_MAP, default=1.0)
      .cast(pl.Float32)
      .alias("sender_entity_type_score"),

    pl.col("receiver_entity_type")
      .replace(ENTITY_TYPE_SCORE_MAP, default=1.0)
      .cast(pl.Float32)
      .alias("receiver_entity_type_score"),
])


In [24]:
# -------------------------
# (F) High risk bank flags
# -------------------------
q_feat = q_feat.with_columns([
    pl.col("Sender_Bank_Name").is_in(HIGH_RISK_SENDER_BANKS).alias("high_risk_sender_bank_flag"),
    pl.col("Receiver_Bank_Name").is_in(HIGH_RISK_RECEIVER_BANKS).alias("high_risk_receiver_bank_flag"),
])

In [25]:
# 확인하고 싶은 핵심 피처만 뽑아서 head 5
inspect_cols = [
    # timestamp 관련
    "Timestamp", "ts", "hour", "day_of_week", "is_weekend", "is_dawn", "timeofday_bucket",

    # amount 관련
    "Amount_Paid_USD", "log_amount_paid_usd",
    "Amount_Received_USD", "log_amount_received_usd",
    "is_round_1000_paid",

    # payment format
    "Payment Format",
    "is_ach", "is_bitcoin_fmt", "is_crypto_transfer",
    "is_high_value_ach", "payment_method_risk", "risk_x_log_paid",

    # entity / bank
    "Sender_Entity", "sender_entity_type", "sender_entity_type_score",
    "Sender_Bank_Name", "high_risk_sender_bank_flag",

    # label / split
    "Is Laundering", "split"
]

q_feat.select(inspect_cols).head(5).collect()

Timestamp,ts,hour,day_of_week,is_weekend,is_dawn,timeofday_bucket,Amount_Paid_USD,log_amount_paid_usd,Amount_Received_USD,log_amount_received_usd,is_round_1000_paid,Payment Format,is_ach,is_bitcoin_fmt,is_crypto_transfer,is_high_value_ach,payment_method_risk,risk_x_log_paid,Sender_Entity,sender_entity_type,sender_entity_type_score,Sender_Bank_Name,high_risk_sender_bank_flag,Is Laundering,split
str,datetime[μs],i16,i8,bool,bool,str,f32,f64,f32,f64,bool,str,bool,bool,bool,bool,f32,f64,str,str,f32,str,bool,i64,str
"""2022/09/01 00:00""",2022-09-01 00:00:00,0,4,false,true,"""dawn""",589.366821,6.380744,589.366821,6.380744,false,"""Reinvestment""",false,false,false,false,1.0,6.380744,"""Partnership #81036""","""Partnership""",1.0,"""France Bank #1654""",false,0,"""train"""
"""2022/09/01 00:00""",2022-09-01 00:00:00,0,4,false,true,"""dawn""",11.48,2.524127,11.48,2.524127,false,"""Reinvestment""",false,false,false,false,1.0,2.524127,"""Sole Proprietorship #51143""","""Sole Proprietorship""",1.5,"""Hilltop Cooperative Bank""",false,0,"""train"""
"""2022/09/01 00:00""",2022-09-01 00:00:00,0,4,false,true,"""dawn""",1838.040039,7.516999,1838.040039,7.516999,false,"""Reinvestment""",false,false,false,false,1.0,7.516999,"""Sole Proprietorship #30920""","""Sole Proprietorship""",1.5,"""Flagstone Thrift""",false,0,"""train"""
"""2022/09/01 00:00""",2022-09-01 00:00:00,0,4,false,true,"""dawn""",3290.0,8.098947,3290.0,8.098947,false,"""ACH""",true,false,false,false,3.0,24.29684,"""Sole Proprietorship #18906""","""Sole Proprietorship""",1.5,"""Savings Bank of Columbus""",false,0,"""train"""
"""2022/09/01 00:00""",2022-09-01 00:00:00,0,4,false,true,"""dawn""",1494.589966,7.310276,1494.589966,7.310276,false,"""Reinvestment""",false,false,false,false,1.0,7.310276,"""Corporation #8424""","""Corporation""",2.0,"""Bank of Philadelphia""",false,0,"""train"""


# 9) 모델에 안 넣을 컬럼 drop    

In [26]:
DROP_COLS_V1 = [
    # raw time / ids
    "Timestamp", "ts", "bucket_ts", "bucket_ts_str",

    # account / bank / entity identifiers
    # "From Account", "To Account",               -> baseline-ml-v2에서는 주석 해제(계좌번호 드랍)
    # "From Bank", "To Bank",
    # "Sender_Bank_Name", "Receiver_Bank_Name",
    # "Sender_Entity", "Receiver_Entity",

    # intermediate entity cols
    "sender_entity_type", "receiver_entity_type",

    # raw amount columns
    "Amount Paid", "Amount Received",
    # "Amount_Paid_USD", "Amount_Received_USD",

    # currency
    # "Receiving Currency", "Payment Currency"

    "sender_node", "receiver_node",
]

q_model = q_feat.drop([c for c in DROP_COLS_V1 if c in q_feat.columns])

In [27]:
# drop 전 51
print(len(q_model.columns))
print(q_model.columns)

43
['From Bank', 'From Account', 'To Bank', 'To Account', 'Receiving Currency', 'Payment Currency', 'Payment Format', 'Is Laundering', 'Amount_Received_USD', 'Amount_Paid_USD', 'Sender_Entity', 'Sender_Bank_Name', 'Receiver_Entity', 'Receiver_Bank_Name', 'split', 'hour', 'day_of_week', 'is_weekend', 'ts_day', 'is_dawn', 'timeofday_bucket', 'log_amount_paid_usd', 'log_amount_received_usd', 'is_round_1000_paid', 'is_round_1000_received', 'is_round_10000_paid', 'is_ach', 'is_cheque', 'is_bitcoin_fmt', 'is_cash', 'is_credit_card', 'is_wire', 'is_reinvestment', 'is_crypto_transfer', 'is_high_value_ach', 'payment_method_risk', 'risk_x_log_paid', 'ach_x_log_paid', 'btc_x_log_paid', 'sender_entity_type_score', 'receiver_entity_type_score', 'high_risk_sender_bank_flag', 'high_risk_receiver_bank_flag']


In [28]:
FLOAT64_COLS = [
    "log_amount_paid_usd",
    "log_amount_received_usd",
    "risk_x_log_paid",
    "ach_x_log_paid",
    "btc_x_log_paid",
]

q_model = q_model.with_columns([
    pl.col(c).cast(pl.Float32) for c in FLOAT64_COLS
])

In [29]:
q_model.schema

Schema([('From Bank', Int64),
        ('From Account', String),
        ('To Bank', Int64),
        ('To Account', String),
        ('Receiving Currency', String),
        ('Payment Currency', String),
        ('Payment Format', String),
        ('Is Laundering', Int64),
        ('Amount_Received_USD', Float32),
        ('Amount_Paid_USD', Float32),
        ('Sender_Entity', String),
        ('Sender_Bank_Name', String),
        ('Receiver_Entity', String),
        ('Receiver_Bank_Name', String),
        ('split', String),
        ('hour', Int16),
        ('day_of_week', Int8),
        ('is_weekend', Boolean),
        ('ts_day', Date),
        ('is_dawn', Boolean),
        ('timeofday_bucket', String),
        ('log_amount_paid_usd', Float32),
        ('log_amount_received_usd', Float32),
        ('is_round_1000_paid', Boolean),
        ('is_round_1000_received', Boolean),
        ('is_round_10000_paid', Boolean),
        ('is_ach', Boolean),
        ('is_cheque', Boolean),
        ('i

In [30]:
#“이 컬럼이 numeric / categorical인지” 빠르게 나누기
schema = q_model.schema

numeric_cols = [
    c for c, d in schema.items()
    if d in (
        pl.Int8, pl.Int16, pl.Int32, pl.Int64,
        pl.Float32, pl.Float64,
        pl.Boolean,   # ✅ 추가
    )
]

categorical_cols = [
    c for c, d in schema.items()
    if d in (pl.Utf8, pl.Categorical)
]

print("Numeric:", numeric_cols)
print("Categorical:", categorical_cols)

Numeric: ['From Bank', 'To Bank', 'Is Laundering', 'Amount_Received_USD', 'Amount_Paid_USD', 'hour', 'day_of_week', 'is_weekend', 'is_dawn', 'log_amount_paid_usd', 'log_amount_received_usd', 'is_round_1000_paid', 'is_round_1000_received', 'is_round_10000_paid', 'is_ach', 'is_cheque', 'is_bitcoin_fmt', 'is_cash', 'is_credit_card', 'is_wire', 'is_reinvestment', 'is_crypto_transfer', 'is_high_value_ach', 'payment_method_risk', 'risk_x_log_paid', 'ach_x_log_paid', 'btc_x_log_paid', 'sender_entity_type_score', 'receiver_entity_type_score', 'high_risk_sender_bank_flag', 'high_risk_receiver_bank_flag']
Categorical: ['From Account', 'To Account', 'Receiving Currency', 'Payment Currency', 'Payment Format', 'Sender_Entity', 'Sender_Bank_Name', 'Receiver_Entity', 'Receiver_Bank_Name', 'split', 'timeofday_bucket']


# 10) 마지막에 pandas로 collect

In [31]:
split_counts = (
    q_feat.group_by("split")
          .agg(pl.count().alias("rows"))
          .collect()
)
print(split_counts)

shape: (3, 2)
┌───────┬──────────┐
│ split ┆ rows     │
│ ---   ┆ ---      │
│ str   ┆ u32      │
╞═══════╪══════════╡
│ test  ┆ 6378958  │
│ train ┆ 19139731 │
│ val   ┆ 6379980  │
└───────┴──────────┘


In [32]:
label_dist = (
    q_feat.group_by(["split", "Is Laundering"])
          .agg(pl.count().alias("rows"))
          .sort(["split", "Is Laundering"])
          .collect()
)
print(label_dist)

shape: (6, 3)
┌───────┬───────────────┬──────────┐
│ split ┆ Is Laundering ┆ rows     │
│ ---   ┆ ---           ┆ ---      │
│ str   ┆ i64           ┆ u32      │
╞═══════╪═══════════════╪══════════╡
│ test  ┆ 0             ┆ 6368318  │
│ test  ┆ 1             ┆ 10640    │
│ train ┆ 0             ┆ 19124195 │
│ train ┆ 1             ┆ 15536    │
│ val   ┆ 0             ┆ 6370926  │
│ val   ┆ 1             ┆ 9054     │
└───────┴───────────────┴──────────┘


In [33]:
# =========================================================
# sklearn로 넘기기 위한 "컬럼만" 선택 후 collect
# =========================================================
label_col = "Is Laundering"
META_COLS = ["ts_day"]  # KPI용 메타

feature_cols = [
    c for c in q_model.columns
    if c not in (["split", label_col] + META_COLS)
]

q_train = (
    q_model.filter(pl.col("split") == "train")
          .select(feature_cols + [label_col] + META_COLS)
)
train_df = q_train.collect().to_pandas()

q_val = (
    q_model.filter(pl.col("split") == "val")
          .select(feature_cols + [label_col] + META_COLS)
)
val_df   = q_val.collect().to_pandas()

q_test = (
    q_model.filter(pl.col("split") == "test")
          .select(feature_cols + [label_col] + META_COLS)
)
test_df  = q_test.collect().to_pandas()

print(train_df.shape, val_df.shape, test_df.shape)

(19139731, 42) (6379980, 42) (6378958, 42)


# 11) Top-k 로직(하루 기준/누적 기준)

In [34]:
# Top-k 로직 교체용 함수:“k개 적발하려면 몇 건을 봐야 하는지”를 계산하는 함수
def workload_to_find_k_positives(y_true, y_score, k_pos):
    """
    목표: true positive를 k_pos개 '찾기' 위해
         상위 점수부터 몇 건(N)을 조사해야 하는지 계산

    Returns:
      N_required: 필요한 조사 건수
      precision:  k_found / N_required
      recall:     k_found / total_pos
      f1:         f1 (top-N을 양성으로 가정한 경우)
      k_found:    실제로 찾은 TP 수 (total_pos < k_pos면 total_pos)
    """
    y_true = np.asarray(y_true).astype(int)
    y_score = np.asarray(y_score)

    total_pos = y_true.sum()
    if total_pos == 0:
        return 0, 0.0, 0.0, 0.0, 0

    # 점수 내림차순 정렬
    order = np.argsort(-y_score)
    y_sorted = y_true[order]

    # 누적 TP
    cum_tp = np.cumsum(y_sorted)

    target = min(k_pos, int(total_pos))

    # cum_tp >= target 되는 첫 index
    idx = np.searchsorted(cum_tp, target, side="left")
    N_required = int(idx + 1)  # index -> count

    k_found = int(cum_tp[idx])  # 보통 target과 같음(동점/중복 없으면)

    precision = k_found / (N_required + 1e-12)
    recall = k_found / (total_pos + 1e-12)

    # top-N을 양성으로 보면: TP=k_found, FP=N-TP, FN=total_pos-TP
    fp = N_required - k_found
    fn = total_pos - k_found
    f1 = (2 * k_found) / (2 * k_found + fp + fn + 1e-12)

    return N_required, precision, recall, f1, k_found

In [35]:
def per_day_workload_summary(df, day_col, label_col, score_col, k_pos_list=(30,50,100,200)):
    """
    df: pandas DataFrame with [day_col, label_col, score_col]
    day별로 'k_pos개 적발하기 위해 필요한 조사량 N' 계산 후
    k_pos마다 daily 분포 + mean/median/p90 요약을 반환
    """
    out_rows = []
    for day, g in df.groupby(day_col):
        y = g[label_col].astype(int).values
        s = g[score_col].values
        total_pos = int(y.sum())
        n = len(g)

        for k_pos in k_pos_list:
            N_req, p, r, f1, found = workload_to_find_k_positives(y, s, k_pos)
            out_rows.append({
                "day": day,
                "n_rows": n,
                "pos": total_pos,
                "k_pos": k_pos,
                "N_required": N_req,
                "precision_at_Nk": p,
                "recall_at_Nk": r,
                "f1_at_Nk": f1,
                "found_tp": found,
                "target_k": min(k_pos, total_pos),
                "hit_target": (found >= min(k_pos, total_pos)) and (total_pos > 0)
            })

    daily = pd.DataFrame(out_rows)

    # day에 pos=0이면 N_required=0이 되고 의미가 약하니,
    # 운영 KPI로는 "pos>0인 day"만 요약하는게 보통 더 깔끔함
    daily_pos = daily[daily["pos"] > 0].copy()

    def summarize(sub):
        # N_required는 "작을수록 좋음"
        return pd.Series({
            "days": sub["day"].nunique(),
            "mean_N_required": sub["N_required"].mean(),
            "median_N_required": sub["N_required"].median(),
            "p90_N_required": sub["N_required"].quantile(0.90),
            "mean_precision": sub["precision_at_Nk"].mean(),
            "median_precision": sub["precision_at_Nk"].median(),
            "p90_precision": sub["precision_at_Nk"].quantile(0.90),
        })

    summary = (
        daily_pos
        .groupby("k_pos", as_index=False)
        .apply(summarize)
        .reset_index(drop=True)
    )

    return daily, summary

In [36]:
def log_per_day_kpi_to_wandb(split_name, df_raw_with_day, y_true, y_score, k_list=(30,50,100,200)):
    """
    split_name: "val" or "test"
    df_raw_with_day: pandas DF that includes column "ts_day" aligned with y_true/y_score index
    y_true/y_score: numpy-like aligned arrays
    """
    tmp = pd.DataFrame({
        "ts_day": df_raw_with_day["ts_day"].values,
        "label": np.asarray(y_true).astype(int),
        "score": np.asarray(y_score),
    })

    daily, summary = per_day_workload_summary(
        tmp,
        day_col="ts_day",
        label_col="label",
        score_col="score",
        k_pos_list=k_list
    )

    # (A) 요약 로그: k별 mean/median/p90
    for _, row in summary.iterrows():
        k = int(row["k_pos"])
        wandb.log({
            f"{split_name}_perday_find_{k}_mean_N": float(row["mean_N_required"]),
            f"{split_name}_perday_find_{k}_median_N": float(row["median_N_required"]),
            f"{split_name}_perday_find_{k}_p90_N": float(row["p90_N_required"]),
            f"{split_name}_perday_find_{k}_mean_precision": float(row["mean_precision"]),
            f"{split_name}_perday_find_{k}_median_precision": float(row["median_precision"]),
            f"{split_name}_perday_find_{k}_p90_precision": float(row["p90_precision"]),
            f"{split_name}_perday_days_with_pos_{k}": int(row["days"]),
        })

    # (B) 분포 확인용 테이블(너무 크면 상위 일부만)
    # day*k 만큼 행이 생김 -> 기간 짧으면 OK, 길면 샘플링/요약만 남기기
    table = wandb.Table(dataframe=daily.head(5000))
    wandb.log({f"{split_name}_perday_workload_table": table})

In [37]:
def score_bin_distribution(y_true, y_score, n_bins=10, score_scale=1000):
    """
    score를 0~score_scale로 스케일링한 뒤,
    bin별 label(0/1) 건수 집계.
    항상 normal_cnt / laundering_cnt 컬럼이 존재하도록 보정.
    """
    y = np.asarray(y_true).astype(int)
    s = np.asarray(y_score)

    s_scaled = np.clip(s * score_scale, 0, score_scale)
    bins = np.linspace(0, score_scale, n_bins + 1)

    # 0..n_bins-1
    bin_idx = np.digitize(s_scaled, bins, right=False) - 1
    bin_idx = np.clip(bin_idx, 0, n_bins - 1)

    df = pd.DataFrame({"bin": bin_idx, "label": y})

    # label별 count (항상 0/1 둘 다 컬럼을 만들도록 reindex)
    cnt = (
        df.groupby(["bin", "label"])
          .size()
          .unstack(fill_value=0)
          .reindex(columns=[0, 1], fill_value=0)   # ✅ 핵심
          .rename(columns={0: "normal_cnt", 1: "laundering_cnt"})
          .reset_index()
    )

    # bin_range 붙이기
    cnt["bin_range"] = cnt["bin"].apply(lambda i: f"{int(bins[i])}-{int(bins[i+1])}")

    # 보기 좋게 정렬
    cnt = cnt[["bin", "bin_range", "normal_cnt", "laundering_cnt"]]
    return cnt

# 12-1) W&B Sweeps용 train/eval 함수 정의(xgboost)

In [38]:
def train_eval_sweep():
    """
    W&B agent가 호출하는 sweep train function.
    - run 안에서 transformer fit은 train만
    - val/test는 transform만 (누수 방지)
    - scale_pos_weight sweep
    - val 기준으로 sweep 최적화 metric 로깅
    - PR curve / CM / Top-K는 val/test 각각 로깅
    - XGB feature importance(gain) 로깅 (변환 후 feature name)
    """
    run = wandb.init()
    config = wandb.config

    # =========================
    # (0) RAW -> X/y 분리
    # =========================
    LABEL_COL = "Is Laundering"

    # train/val/test는 "이미 time split 완료된 pandas df"라고 가정
    # (run 밖에서 만들어둔 걸 그대로 씀)
    X_train_raw = train_df.drop(columns=[LABEL_COL, "ts_day"])
    y_train     = train_df[LABEL_COL].astype(int)

    X_val_raw   = val_df.drop(columns=[LABEL_COL, "ts_day"])
    y_val       = val_df[LABEL_COL].astype(int)

    X_test_raw  = test_df.drop(columns=[LABEL_COL, "ts_day"])
    y_test      = test_df[LABEL_COL].astype(int)

    # =========================
    # (1) 컬럼 타입 분리 (run마다 동일하지만 안전하게 여기서)
    # =========================
    categorical_features = X_train_raw.select_dtypes(include=["object", "category", "string"]).columns.tolist()
    numerical_features   = [c for c in X_train_raw.columns if c not in categorical_features]

    # =========================
    # (2) Transformer 정의 & fit/transform (누수 방지 핵심)
    # =========================
    num_pipe = Pipeline(steps=[
        ("imputer", SimpleImputer(strategy="median")),
        # XGBoost면 보통 스케일링 생략 가능 (원하면 config로 켜도 됨)
    ])

    cat_pipe = Pipeline(steps=[
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("enc", OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1)),
    ])

    transformer = ColumnTransformer(
        transformers=[
            ("num", num_pipe, numerical_features),
            ("cat", cat_pipe, categorical_features),
        ],
        remainder="drop",
        verbose_feature_names_out=False,  # get_feature_names_out 깔끔하게
    )

    # ✅ fit은 train만
    X_train = transformer.fit_transform(X_train_raw)
    # ✅ val/test는 transform만
    X_val   = transformer.transform(X_val_raw)
    X_test  = transformer.transform(X_test_raw)

    # 변환 후 feature name 확보 (importance 로깅용)
    try:
        feature_names = transformer.get_feature_names_out()
        feature_names = [str(x) for x in feature_names]
    except Exception:
        feature_names = [f"f{i}" for i in range(X_train.shape[1])]

    # =========================
    # (A) 모델 생성 (sweep 파라미터)
    # =========================
    xgb_model = XGBClassifier(
        n_estimators=config.n_estimators,
        max_depth=config.max_depth,
        learning_rate=config.learning_rate,
        subsample=config.subsample,
        colsample_bytree=config.colsample_bytree,
        reg_lambda=config.reg_lambda,
        reg_alpha=config.reg_alpha,
        min_child_weight=config.min_child_weight,
        gamma=config.gamma,
        scale_pos_weight=config.scale_pos_weight,   # ✅ sweep
        eval_metric="logloss",
        random_state=42,
        n_jobs=-1
    )

    # =========================
    # (B) 학습
    # =========================
    xgb_model.fit(X_train, y_train)

    # =========================
    # (C) 평가 (val/test 둘 다)
    # =========================
    def safe_auc_metrics(y_true, y_score):
        if len(np.unique(y_true)) <= 1:
            return np.nan, np.nan
        return roc_auc_score(y_true, y_score), average_precision_score(y_true, y_score)

    def topk_metrics(y_true, y_score, k):
        k = min(k, len(y_true))
        idx = np.argsort(-y_score)[:k]
        y_pred = np.zeros_like(y_true)
        y_pred[idx] = 1
        p = precision_score(y_true, y_pred, zero_division=0)
        r = recall_score(y_true, y_pred, zero_division=0)
        f = f1_score(y_true, y_pred, zero_division=0)
        return p, r, f

    def threshold_at_recall(y_true, y_score, desired_recall=0.90):
        prec, rec, thr = precision_recall_curve(y_true, y_score)
        if len(thr) == 0:
            return 0.5, prec, rec, thr
        closest_i = np.argmin(np.abs(rec[1:] - desired_recall))
        return float(thr[closest_i]), prec, rec, thr

    # -------------------------
    # (C-1) val / test 확률 예측
    # -------------------------
    prob_val  = xgb_model.predict_proba(X_val)[:, 1]
    prob_test = xgb_model.predict_proba(X_test)[:, 1]

    val_roc_auc,  val_auprc  = safe_auc_metrics(y_val, prob_val)
    test_roc_auc, test_auprc = safe_auc_metrics(y_test, prob_test)

    # ✅ sweep 최적화 metric은 val 기준으로 통일
    wandb.log({
        "val_roc_auc":  val_roc_auc,
        "val_auprc":    val_auprc,
        "test_roc_auc": test_roc_auc,
        "test_auprc":   test_auprc,
        "n_train": len(y_train),
        "pos_train": int(np.sum(y_train)),
        "pos_val": int(np.sum(y_val)),
        "pos_test": int(np.sum(y_test)),
    })

    # -------------------------
    # (C-2) PR Curve 로깅 (val/test 각각)
    # -------------------------
    # PR curve 로깅 (val/test)
    prec_v, rec_v, _ = precision_recall_curve(y_val, prob_val)
    fig_pr_v = plt.figure()
    plt.plot(rec_v, prec_v)
    plt.xlabel("Recall"); plt.ylabel("Precision")
    plt.title(f"VAL PR Curve (AUPRC={val_auprc:.4f})")
    wandb.log({"val_pr_curve": wandb.Image(fig_pr_v)})
    plt.close(fig_pr_v)

    prec_t, rec_t, _ = precision_recall_curve(y_test, prob_test)
    fig_pr_t = plt.figure()
    plt.plot(rec_t, prec_t)
    plt.xlabel("Recall"); plt.ylabel("Precision")
    plt.title(f"TEST PR Curve (AUPRC={test_auprc:.4f})")
    wandb.log({"test_pr_curve": wandb.Image(fig_pr_t)})
    plt.close(fig_pr_t)

    # -------------------------
    # (C-3) Top-k
    # -------------------------
    # "K TP 찾기 위한 조사량" 로깅 (val/test 각각)
    for k_pos in [450, 750, 1500, 3000]:
        N_v, p_v, r_v, f_v, found_v = workload_to_find_k_positives(y_val.values, prob_val, k_pos)
        N_t, p_t, r_t, f_t, found_t = workload_to_find_k_positives(y_test.values, prob_test, k_pos)

        wandb.log({
            # VAL
            f"val_find_{k_pos}_N_required": N_v,
            f"val_find_{k_pos}_precision": p_v,
            f"val_find_{k_pos}_recall":    r_v,
            f"val_find_{k_pos}_f1":        f_v,
            f"val_find_{k_pos}_found_tp":  found_v,

            # TEST
            f"test_find_{k_pos}_N_required": N_t,
            f"test_find_{k_pos}_precision": p_t,
            f"test_find_{k_pos}_recall":    r_t,
            f"test_find_{k_pos}_f1":        f_t,
            f"test_find_{k_pos}_found_tp":  found_t,
        })

    # per-day KPI 로깅 (VAL/TEST 각각)
    # 전제: val_df/test_df에 ts_day 컬럼 존재
    log_per_day_kpi_to_wandb("val",  val_df,  y_val.values,  prob_val,  k_list=(30,50,100,200))
    log_per_day_kpi_to_wandb("test", test_df, y_test.values, prob_test, k_list=(30,50,100,200))

    dist_val  = score_bin_distribution(y_val.values,  prob_val,  n_bins=10, score_scale=1000)
    dist_test = score_bin_distribution(y_test.values, prob_test, n_bins=10, score_scale=1000)
    wandb.log({
        "val_score_bin_table":  wandb.Table(dataframe=dist_val),
        "test_score_bin_table": wandb.Table(dataframe=dist_test),
    })

    # -------------------------
    # (C-4) threshold@recall (val에서 threshold 선택 → test에 적용)
    # -------------------------
    desired_recall = 0.90
    chosen_thr, _, _, _ = threshold_at_recall(y_val, prob_val, desired_recall)

    # ✅ thr 기반 예측(한 번만)
    y_val_pred  = (prob_val  >= chosen_thr).astype(int)
    y_test_pred = (prob_test >= chosen_thr).astype(int)

    # ✅ CM (thr 기반)
    cm_val  = confusion_matrix(y_val,  y_val_pred)
    cm_test = confusion_matrix(y_test, y_test_pred)

    fig_cm_v = plt.figure(figsize=(6,4))
    sns.heatmap(cm_val, annot=True, fmt="d", cmap="Blues")
    plt.xlabel("Predicted"); plt.ylabel("True")
    plt.title(f"VAL CM @thr={chosen_thr:.4f}")
    wandb.log({"val_confusion_matrix": wandb.Image(fig_cm_v)})
    plt.close(fig_cm_v)

    fig_cm_t = plt.figure(figsize=(6,4))
    sns.heatmap(cm_test, annot=True, fmt="d", cmap="Blues")
    plt.xlabel("Predicted"); plt.ylabel("True")
    plt.title(f"TEST CM @thr={chosen_thr:.4f}")
    wandb.log({"test_confusion_matrix": wandb.Image(fig_cm_t)})
    plt.close(fig_cm_t)

    # =========================
    # (C-4-2) Precision / Recall / F1
    # =========================

    # (A) chosen_thr 기준 (운영)
    val_precision_thr  = precision_score(y_val,  y_val_pred,  zero_division=0)
    val_recall_thr     = recall_score(y_val,     y_val_pred,  zero_division=0)
    val_f1_thr         = f1_score(y_val,         y_val_pred,  zero_division=0)

    test_precision_thr = precision_score(y_test, y_test_pred, zero_division=0)
    test_recall_thr    = recall_score(y_test,    y_test_pred, zero_division=0)
    test_f1_thr        = f1_score(y_test,        y_test_pred, zero_division=0)

    # (B) 0.5 기준 (참고)
    thr_05 = 0.5
    y_val_pred_05  = (prob_val  >= thr_05).astype(int)
    y_test_pred_05 = (prob_test >= thr_05).astype(int)

    val_precision_05  = precision_score(y_val,  y_val_pred_05,  zero_division=0)
    val_recall_05     = recall_score(y_val,     y_val_pred_05,  zero_division=0)
    val_f1_05         = f1_score(y_val,         y_val_pred_05,  zero_division=0)

    test_precision_05 = precision_score(y_test, y_test_pred_05, zero_division=0)
    test_recall_05    = recall_score(y_test,    y_test_pred_05, zero_division=0)
    test_f1_05        = f1_score(y_test,        y_test_pred_05, zero_division=0)

    # ✅ 로그(한 번에)
    wandb.log({
        "desired_recall_for_thr": float(desired_recall),
        "chosen_threshold": float(chosen_thr),

        "val_precision_thr":  float(val_precision_thr),
        "val_recall_thr":     float(val_recall_thr),
        "val_f1_thr":         float(val_f1_thr),
        "test_precision_thr": float(test_precision_thr),
        "test_recall_thr":    float(test_recall_thr),
        "test_f1_thr":        float(test_f1_thr),

        "val_precision_thr_0.5":  float(val_precision_05),
        "val_recall_thr_0.5":     float(val_recall_05),
        "val_f1_thr_0.5":         float(val_f1_05),
        "test_precision_thr_0.5": float(test_precision_05),
        "test_recall_thr_0.5":    float(test_recall_05),
        "test_f1_thr_0.5":        float(test_f1_05),
    })

    # =========================
    # (C-5) Feature Importance (gain)
    # =========================
    booster = xgb_model.get_booster()
    gain_dict = booster.get_score(importance_type="gain")

    if len(gain_dict) > 0:
        imp_rows = []
        for k, v in gain_dict.items():
            try:
                idx = int(k[1:])
            except Exception:
                continue
            name = feature_names[idx] if idx < len(feature_names) else k
            imp_rows.append((name, float(v)))

        imp_rows.sort(key=lambda x: x[1], reverse=True)

        # (1) ✅ 전체 table 로깅
        table_all = wandb.Table(columns=["feature", "gain"])
        for name, gain in imp_rows:
            table_all.add_data(name, gain)
        wandb.log({"feature_importance_gain_table_all": table_all})

        # (2) bar는 top만
        topN_bar = int(getattr(config, "topn_importance", 50)) if hasattr(config, "topn_importance") else 50
        imp_top = imp_rows[:topN_bar]

        fig_imp = plt.figure(figsize=(8, 10))
        names = [x[0] for x in imp_top][::-1]
        vals  = [x[1] for x in imp_top][::-1]
        plt.barh(names, vals)
        plt.xlabel("Gain")
        plt.title(f"Top-{len(imp_top)} Feature Importance (Gain)")
        wandb.log({"feature_importance_gain_bar_top": wandb.Image(fig_imp)})
        plt.close(fig_imp)

    # =========================
    # (D) 콘솔 리포트 (test)
    # =========================
    print("=== Sweep Report ===")
    print("VAL  ROC-AUC:", val_roc_auc,  "VAL  AUPRC:", val_auprc)
    print("TEST ROC-AUC:", test_roc_auc, "TEST AUPRC:", test_auprc)
    print("Chosen thr(from VAL):", chosen_thr)
    print(classification_report(y_test, y_test_pred, zero_division=0))

    # =========================
    # (E) Artifacts 저장/업로드
    # =========================
    os.makedirs("artifacts_out", exist_ok=True)

    preproc_path = "artifacts_out/transformer.joblib"
    joblib.dump(transformer, preproc_path)

    model_joblib_path = "artifacts_out/aml_model.joblib"
    joblib.dump(xgb_model, model_joblib_path)

    model_pkl_path = "artifacts_out/aml_model.pkl"
    with open(model_pkl_path, "wb") as f:
        pickle.dump(xgb_model, f)

    # (선택) 데이터 샘플 저장
    data_sample_path = "artifacts_out/train_sample.csv"
    try:
        sample_n = 2000
        X_train_raw_sample = X_train_raw.head(sample_n).copy()
        y_train_sample = y_train.loc[X_train_raw_sample.index].copy()
        X_train_raw_sample[LABEL_COL] = y_train_sample
        X_train_raw_sample.to_csv(data_sample_path, index=False)
        include_sample = True
    except Exception:
        include_sample = False

    art = wandb.Artifact(
        name="aml_bundle",
        type="model",
        description="Transformer(train-only fit) + XGB(scale_pos_weight sweep) + optional data sample"
    )
    art.add_file(preproc_path)
    art.add_file(model_joblib_path)
    art.add_file(model_pkl_path)
    if include_sample:
        art.add_file(data_sample_path)

    wandb.log_artifact(art)

    run.finish()

# 12-2) Sweep Config 정의 + 실행

In [39]:
sweep_config = {
    "method": "bayes",
    "metric": {"name": "val_auprc", "goal": "maximize"},  # ★ 수정
    "parameters": {
        "n_estimators": {"values": [600]},
        "max_depth": {"values": [8]},
        "learning_rate": {"values": [0.1]},
        "subsample": {"values": [0.7]},
        "colsample_bytree": {"values": [1.0]},
        "reg_lambda": {"values": [5.0]},
        "reg_alpha": {"values": [0.5]},
        "min_child_weight": {"values": [10]},
        "gamma": {"values": [0.5]},
        "scale_pos_weight": {"values": [1.0]},  # ★ 추가
    }
}

# (프로젝트/엔티티 이름은 팀 기준으로 정해줘)
sweep_id = wandb.sweep(sweep_config, project="eungyulwon")

# count: 몇 번의 실험(run)을 돌릴지
wandb.agent(sweep_id, function=train_eval_sweep, count=1)

Create sweep with ID: svpv850k
Sweep URL: https://wandb.ai/0326byeol-korea-ac-kr/eungyulwon/sweeps/svpv850k


wandb: Agent Starting Run: owq43yud with config:
wandb: 	colsample_bytree: 1
wandb: 	gamma: 0.5
wandb: 	learning_rate: 0.1
wandb: 	max_depth: 8
wandb: 	min_child_weight: 10
wandb: 	n_estimators: 600
wandb: 	reg_alpha: 0.5
wandb: 	reg_lambda: 5
wandb: 	scale_pos_weight: 1
wandb: 	subsample: 0.7
wandb: [wandb.login()] Loaded credentials for https://api.wandb.ai from /root/.netrc.


=== Sweep Report ===
VAL  ROC-AUC: 0.9799227482784912 VAL  AUPRC: 0.31476612142923177
TEST ROC-AUC: 0.984481708271893 TEST AUPRC: 0.44811858920485814
Chosen thr(from VAL): 0.001552874338813126
              precision    recall  f1-score   support

           0       1.00      0.93      0.97   6368318
           1       0.02      0.91      0.04     10640

    accuracy                           0.93   6378958
   macro avg       0.51      0.92      0.50   6378958
weighted avg       1.00      0.93      0.96   6378958



chosen_threshold,▁
desired_recall_for_thr,▁
n_train,▁
pos_test,▁
pos_train,▁
pos_val,▁
test_auprc,▁
test_f1_thr,▁
test_f1_thr_0.5,▁
test_find_1500_N_required,▁
+108,...


In [40]:
# # v1 best run hyperparameters 주입, count = 1
# sweep_config = {
#     "method": "bayes",
#     "metric": {"name": "val_auprc", "goal": "maximize"},  # ★ 수정
#     "parameters": {
#         "n_estimators": {"values": [600]},
#         "max_depth": {"values": [8]},
#         "learning_rate": {"values": [0.03]},
#         "subsample": {"values": [0.9]},
#         "colsample_bytree": {"values": [1.0]},
#         "reg_lambda": {"values": [1.0]},
#         "reg_alpha": {"values": [0.1]},
#         "min_child_weight": {"values": [5]},
#         "gamma": {"values": [0.5]},
#         "scale_pos_weight": {"values": [1]},  # ★ 추가
#     }
# }

# # (프로젝트/엔티티 이름은 팀 기준으로 정해줘)
# sweep_id = wandb.sweep(sweep_config, project="eungyulwon")

# # count: 몇 번의 실험(run)을 돌릴지
# wandb.agent(sweep_id, function=train_eval_sweep, count=1)

In [41]:
# # v2 best run hyperparameters 주입, count = 1
# sweep_config = {
#     "method": "bayes",
#     "metric": {"name": "val_auprc", "goal": "maximize"},  # ★ 수정
#     "parameters": {
#         "n_estimators": {"values": [600]},
#         "max_depth": {"values": [8]},
#         "learning_rate": {"values": [0.1]},
#         "subsample": {"values": [0.7]},
#         "colsample_bytree": {"values": [1.0]},
#         "reg_lambda": {"values": [5.0]},
#         "reg_alpha": {"values": [0.5]},
#         "min_child_weight": {"values": [10]},
#         "gamma": {"values": [0.5]},
#         "scale_pos_weight": {"values": [1]},  # ★ 추가
#     }
# }

# # (프로젝트/엔티티 이름은 팀 기준으로 정해줘)
# sweep_id = wandb.sweep(sweep_config, project="eungyulwon")

# # count: 몇 번의 실험(run)을 돌릴지
# wandb.agent(sweep_id, function=train_eval_sweep, count=1)

# 15) (선택) Artifacts "불러오기" 예시 (필요할 때만 실행)
#     - W&B에서 best 모델 버전 지정해서 다시 로컬로 가져올 수 있음

In [42]:
# run = wandb.init(project="aml-team-experiments", job_type="inference")
# artifact = run.use_artifact("YOUR_ENTITY/aml-team-experiments/aml_xgb_bundle:latest", type="model")
# artifact_dir = artifact.download()
# loaded_transformer = joblib.load(os.path.join(artifact_dir, "transformer.joblib"))
# loaded_model = joblib.load(os.path.join(artifact_dir, "aml_model.joblib"))
# run.finish()

# 이제 loaded_transformer/loaded_model로 동일 파이프라인 재현 가능